In [1]:
import mido
from matplotlib import pyplot as plt

In [2]:
mido.set_backend('mido.backends.rtmidi')
outport = mido.open_output()
port = mido.open_output(name='foo', virtual=True)

In [6]:
love_me = mido.MidiFile('../../songs/small_db/Love_Me_Do.mid')
bohemian = mido.MidiFile('../../songs/small_db/bohemian_rhapsody.mid')
dream = mido.MidiFile('../../songs/small_db/Dream_On.mid')

midis = [love_me, bohemian, dream]